In [76]:
import os
import numpy as np
import pandas as pd
import librosa
# Replace this with your actual path
preprocessed_dataset_path = r"..\PreProcessedDataSet_for_ML"

n_fft = 400 # 25 ms window 
hop_length = 160 # 10 ms step → 60% overlap


In [77]:
def compute_mfcc(y, sr, n_mfcc, n_fft, hop_length):
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    return mfcc.mean(axis=1), mfcc.std(axis=1)

def compute_zcr(y, n_fft, hop_length):
    zcr = librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length)
    return zcr.mean(), zcr.std()

def compute_rms(y, n_fft, hop_length):
    rms = librosa.feature.rms(y=y, frame_length=n_fft, hop_length=hop_length)
    return rms.mean(), rms.std()

def compute_centroid(y, sr, n_fft, hop_length):
    centroid = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)
    return centroid.mean(), centroid.std()

def compute_bandwidth(y, sr, n_fft, hop_length):
    bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)
    return bandwidth.mean(), bandwidth.std()

def compute_rolloff(y, sr, n_fft, hop_length):
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)
    return rolloff.mean(), rolloff.std()

def compute_chroma(y, sr, n_fft, hop_length):
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)
    return chroma.mean(axis=1), chroma.std(axis=1)

def compute_spectral_flux(y, sr, n_fft, hop_length):
    S = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
    flux = np.sqrt(np.sum(np.diff(S, axis=1)**2, axis=0))
    return flux.mean(), flux.std()


In [ ]:

def extract_features_from_audio(file_path, sr=16000, n_mfcc=13, n_fft=2048, hop_length=512):
    try:
        y, _ = librosa.load(file_path, sr=sr)

        mfcc_mean, mfcc_std = compute_mfcc(y, sr, n_mfcc, n_fft, hop_length)

        zcr_mean, zcr_std = compute_zcr(y, n_fft, hop_length)

        rms_mean, rms_std = compute_rms(y, n_fft, hop_length)

        centroid_mean, centroid_std = compute_centroid(y, sr, n_fft, hop_length)

        bandwidth_mean, bandwidth_std = compute_bandwidth(y, sr, n_fft, hop_length)

        rolloff_mean, rolloff_std = compute_rolloff(y, sr, n_fft, hop_length)

        chroma_mean, chroma_std = compute_chroma(y, sr, n_fft, hop_length)
        
        flux_mean, flux_std = compute_spectral_flux(y, sr, n_fft, hop_length)

        return np.concatenate((
            mfcc_mean, mfcc_std,
            [zcr_mean, zcr_std, rms_mean, rms_std,
             centroid_mean, centroid_std,
             bandwidth_mean, bandwidth_std,
             rolloff_mean, rolloff_std,
             flux_mean, flux_std],
            chroma_mean, chroma_std
        ))

    except Exception as e:
        print(f"Error with {file_path}: {e}")
        return None


In [55]:
features = []
filenames = []

for file in os.listdir(preprocessed_dataset_path):
    if file.lower().endswith(".wav"):
        path = os.path.join(preprocessed_dataset_path, file)
        result = extract_features_from_audio(path)
        if result is not None:
            print(f"features are extracted from : {file}")
            features.append(result)
            filenames.append(file)

# Feature column names

mfcc_cols = [f"mfcc{i+1}_mean" for i in range(13)] + [f"mfcc{i+1}_std" for i in range(13)]

spec_cols = [
    "zcr_mean", "zcr_std",
    "rms_mean", "rms_std",
    "centroid_mean", "centroid_std",
    "bandwidth_mean", "bandwidth_std",
    "rolloff_mean", "rolloff_std",
    "spectral_flux_mean", "spectral_flux_std"
]
chroma_cols = [f"chroma{i+1}_mean" for i in range(12)] + [f"chroma{i+1}_std" for i in range(12)]

columns = ["file_name"] + mfcc_cols + spec_cols + chroma_cols

df = pd.DataFrame(features, columns=mfcc_cols + spec_cols + chroma_cols)
df.insert(0, "file_name", filenames)
df.to_csv(r"Analytics\audio_features.csv", index=False)


features are extracted from : 1001_DFA_ANG_XX.wav
features are extracted from : 1001_DFA_DIS_XX.wav
features are extracted from : 1001_DFA_FEA_XX.wav
features are extracted from : 1001_DFA_HAP_XX.wav
features are extracted from : 1001_DFA_NEU_XX.wav
features are extracted from : 1001_DFA_SAD_XX.wav
features are extracted from : 1001_IEO_ANG_HI.wav
features are extracted from : 1001_IEO_ANG_LO.wav
features are extracted from : 1001_IEO_ANG_MD.wav
features are extracted from : 1001_IEO_DIS_HI.wav
features are extracted from : 1001_IEO_DIS_LO.wav
features are extracted from : 1001_IEO_DIS_MD.wav
features are extracted from : 1001_IEO_FEA_HI.wav
features are extracted from : 1001_IEO_FEA_LO.wav
features are extracted from : 1001_IEO_FEA_MD.wav
features are extracted from : 1001_IEO_HAP_HI.wav
features are extracted from : 1001_IEO_HAP_LO.wav
features are extracted from : 1001_IEO_HAP_MD.wav
features are extracted from : 1001_IEO_NEU_XX.wav
features are extracted from : 1001_IEO_SAD_HI.wav


KeyboardInterrupt: 